In [1]:
import sys
#sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/env/lib/python3.7/site-packages')
sys.path.append('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/analysis/')
from functions import *

In [2]:
def get_resids(data, var, formula):
    import statsmodels.formula.api as smf
    model = smf.ols(var + formula, data=data)
    model = model.fit()
    resid = model.resid
    return resid

def add_resid(data, var, formula):
    data[var + "_resid"] = get_resids(data, var, formula)
    return data

full_formula = '~ interview_age + C(sex)'
no_sex_formula = '~ interview_age'

In [3]:
include = pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/inclusion_ids.csv')

dropped_siblings = pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/dropped_siblings.csv')

participants_full = (pd.read_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/participants.tsv', sep='\t')
                        .assign(ID = lambda x: x['participant_id'].str.replace('sub-', ''),
                                sex = lambda x: np.where(x['sex'] == 1, 'Male', 'Female'))
                        .drop_duplicates('ID', keep='first')
                    )

#participants_site_scanner_software = participants_full[['ID', 'site', 'scanner_model', 'scanner_software']]

participants = participants_full[['ID', 'matched_group']]
participants_sex = participants_full[['ID', 'sex']]
participants = participants.merge(dropped_siblings, on='ID')
participants = participants.merge(include, on='ID')
participants = participants [['ID', 'matched_group']]

In [4]:
sample1 = participants.query('matched_group == 1')
sample2 = participants.query('matched_group == 2')

males = participants_sex.query('sex == "Male"')
females = participants_sex.query('sex == "Female"')

In [5]:
rpath_subcort = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/mrirscor02.txt'


rest_baseline_subcort = readtxt(rpath_subcort)
rest_baseline_subcort['src_subject_id'] = rest_baseline_subcort['src_subject_id'].str.replace('_', '', regex=True)

rest_baseline_subcort_mask = rest_baseline_subcort['eventname'].str.contains('baseline')   
rest_baseline_subcort = rest_baseline_subcort[rest_baseline_subcort_mask]

rest_baseline_subcort = rest_baseline_subcort.drop(['collection_id', 'interview_age', 'mrirscor02_id','dataset_id', 'subjectkey',
                                                    'interview_date', 'eventname','rsfmri_cor_ngd_scs_visitid', 'sex',
                                                    'rsfmri_cor_ngd_scs_tr','rsfmri_cor_ngd_scs_numtrs','rsfmri_cor_ngd_scs_nvols',
                                                    'rsfmri_cor_ngd_scs_stnvols','rsfmri_cor_ngd_scs_stcgnvols','rsfmri_cor_ngd_scs_ntpoints',
                                                    'rsfmri_cor_ngd_scs_meanmn','rsfmri_cor_ngd_scs_maxmn','rsfmri_cor_ngd_scs_meantrans',
                                                    'rsfmri_cor_ngd_scs_maxtrans','rsfmri_cor_ngd_scs_meanrot','rsfmri_cor_ngd_scs_maxrot','collection_title'], axis = 1).rename(columns={"src_subject_id": "ID"})

rest_baseline_subcort.columns = rest_baseline_subcort.columns.str.replace('rsfmri_cor_ngd_', '', regex=True)
rest_baseline_subcort.columns = rest_baseline_subcort.columns.str.replace('_scs_', '_', regex=True) 

rest_baseline_subcort[rest_baseline_subcort.iloc[:, 1:].columns] = rest_baseline_subcort.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')

rest_baseline_subcort = pd.merge(rest_baseline_subcort, participants[['ID']], on='ID')

In [6]:
rest_baseline_subcort.shape

(7525, 248)

In [7]:
#rest_creation = '''
rpath = '/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/abcd_betnet02.txt'

rest_baseline = readtxt(rpath)
rest_baseline['src_subject_id'] = rest_baseline['src_subject_id'].str.replace('_', '', regex=True)

rest_1_mask = rest_baseline['rsfmri_c_ngd_visitid'].str.contains('baseline')   
rest_baseline= rest_baseline[rest_1_mask]

rest_motion = rest_baseline[['src_subject_id', 'rsfmri_c_ngd_tr', 'rsfmri_c_ngd_numtrs',
       'rsfmri_c_ngd_nvols', 'rsfmri_c_ngd_stnvols',
       'rsfmri_c_ngd_stcontignvols', 'rsfmri_c_ngd_ntpoints',
       'rsfmri_c_ngd_meanmotion', 'rsfmri_c_ngd_maxmotion',
       'rsfmri_c_ngd_meantrans', 'rsfmri_c_ngd_maxtrans',
       'rsfmri_c_ngd_meanrot', 'rsfmri_c_ngd_maxrot']].rename(columns={"src_subject_id": "ID"})

rest_baseline = rest_baseline.drop(['collection_id', 'abcd_betnet02_id', 'dataset_id', 'subjectkey','interview_date', 'eventname',
       'rsfmri_c_ngd_visitid', 'rsfmri_c_ngd_tr', 'rsfmri_c_ngd_numtrs',
       'rsfmri_c_ngd_nvols', 'rsfmri_c_ngd_stnvols',
       'rsfmri_c_ngd_stcontignvols', 'rsfmri_c_ngd_ntpoints',
       'rsfmri_c_ngd_meanmotion', 'rsfmri_c_ngd_maxmotion',
       'rsfmri_c_ngd_meantrans', 'rsfmri_c_ngd_maxtrans',
       'rsfmri_c_ngd_meanrot', 'rsfmri_c_ngd_maxrot', 'collection_title'], axis = 1).rename(columns={"src_subject_id": "ID"})

rest_baseline.columns = rest_baseline.columns.str.replace('rsfmri_c_ngd_', '', regex=True)
rest_baseline.columns = rest_baseline.columns.str.replace('_ngd_', '_', regex=True) 

rest_cols_subset = ['ID','interview_age', 'sex', 'ad_ad','ad_cgc','ad_ca','ad_dt','ad_dla',
'ad_fo','ad_n','ad_rspltp','ad_smh','ad_smm','ad_sa','ad_vta','ad_vs','cgc_cgc',
'cgc_ca','cgc_dt','cgc_dla','cgc_fo','cgc_n','cgc_rspltp','cgc_smh','cgc_smm','cgc_sa',
'cgc_vta','cgc_vs','ca_ca','ca_dt','ca_dla','ca_fo','ca_n','ca_rspltp','ca_smh','ca_smm',
'ca_sa','ca_vta','ca_vs','dt_dt','dt_dla','dt_fo','dt_n','dt_rspltp','dt_smh','dt_smm','dt_sa','dt_vta',
'dt_vs','dla_dla','dla_fo','dla_n','dla_rspltp','dla_smh','dla_smm','dla_sa','dla_vta',
'dla_vs','fo_fo','fo_n','fo_rspltp','fo_smh','fo_smm','fo_sa','fo_vta','fo_vs','n_n',
'n_rspltp','n_smh','n_smm','n_sa','n_vta','n_vs','rspltp_rspltp','rspltp_smh','rspltp_smm',
'rspltp_sa','rspltp_vta','rspltp_vs','smh_smh','smh_smm','smh_sa','smh_vta','smh_vs',
'smm_smm','smm_sa','smm_vta','smm_vs','sa_sa','sa_vta','sa_vs','vta_vta','vta_vs','vs_vs']

rest_baseline = rest_baseline[rest_cols_subset]

rest_baseline[rest_cols_subset[3:]] = rest_baseline[rest_cols_subset[3:]].apply(pd.to_numeric, errors='coerce')
rest_baseline['interview_age'] = rest_baseline['interview_age'].apply(pd.to_numeric, errors='coerce')
#rest_baseline = pd.merge(rest_baseline, participants_sex, on = 'ID').dropna().drop_duplicates()
rest_baseline = pd.merge(rest_baseline, participants[['ID']], on='ID')

rest_baseline.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_include.csv')
rest_motion.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_motion_include.csv')

In [8]:
rest_baseline.shape

(7525, 94)

In [9]:
rest_baseline = pd.merge(rest_baseline, rest_baseline_subcort, on ='ID').dropna()


In [10]:
rest_baseline_resid = rest_baseline.copy()

for i in rest_baseline_resid.drop(['ID', 'sex', 'interview_age'], axis=1).columns:
    rest_baseline_resid = add_resid(rest_baseline_resid, i, full_formula)

In [11]:
rest_baseline_resid = pd.concat([rest_baseline_resid[['ID']],rest_baseline_resid.filter(regex='_resid')], axis=1)
rest_baseline_resid.columns = [col.replace('_resid', '') for col in rest_baseline_resid.columns]
#rest_baseline_resid.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/rest/rest_baseline_resid_include.csv')

In [12]:
#rest_cols = rest_baseline_resid.iloc[:, 1:].columns.to_list()
#rest_outliers = get_anoms(rest_baseline_resid, rest_cols)
#print(rest_outliers.groupby('outliers_sum').count().iloc[:, :1])
#rest_outliers = rest_outliers.query('outliers_sum != -3').drop('outliers_sum', axis=1)

In [13]:
sample1_rest = pd.merge(rest_baseline_resid, sample1, on='ID').drop('matched_group', axis=1)
sample2_rest = pd.merge(rest_baseline_resid, sample2, on='ID').drop('matched_group', axis=1)
#sample1_rest= pd.merge(rest_outliers, sample1, on='ID').drop('matched_group', axis=1)
#sample2_rest= pd.merge(rest_outliers, sample2, on='ID').drop('matched_group', axis=1)

sample1_rest.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/sample1_rest_baseline_07282023_include.csv')
sample2_rest.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/sample2_rest_baseline_07282023_include.csv')

In [14]:
full_sample = pd.concat([sample1_rest, sample2_rest])

full_sample.to_csv('/pl/active/banich/studies/Relevantstudies/abcd/data/clustering/subtype_data/baseline/fullsample_rest_baseline_07282023_include.csv')

In [15]:
full_sample

,ID,ad_ad,ad_cgc,ad_ca,ad_dt,ad_dla,ad_fo,ad_n,ad_rspltp,ad_smh,...,vs_vtdclh,vs_crcxrh,vs_thprh,vs_cderh,vs_ptrh,vs_plrh,vs_hprh,vs_agrh,vs_aarh,vs_vtdcrh
0,NDARINV7XVXWCME,-0.001577,-0.045262,-0.032094,0.110062,-0.020885,0.044049,0.042743,0.017627,0.022806,...,0.024247,-0.003212,0.037856,0.014504,-0.136564,-0.008270,-0.059348,-0.083262,-0.012038,-0.092842
1,NDARINV8CNEFJ83,-0.027307,0.035555,-0.062622,-0.082260,0.044518,-0.049984,-0.040557,-0.037724,-0.007616,...,-0.045872,0.039699,-0.016517,-0.043681,-0.043677,0.050746,-0.036145,0.009576,-0.149993,0.030372
2,NDARINV8M2Z9BTA,-0.043190,0.009080,0.023993,-0.009535,0.002787,0.007539,-0.018991,-0.033844,-0.046081,...,-0.007734,0.010744,-0.015998,0.061971,0.012459,-0.038586,0.132450,-0.012906,0.053939,0.004283
3,NDARINVDFY7YYM9,0.028982,-0.004365,-0.044591,0.016543,-0.003571,-0.011947,-0.027667,-0.054711,0.009479,...,-0.013082,-0.016081,-0.010487,0.018995,0.010881,0.094942,0.018040,-0.151601,-0.128813,-0.017876
4,NDARINVE2JXEZ3N,0.083074,0.069749,-0.072002,-0.066413,0.013284,-0.022991,0.016597,-0.066318,-0.088762,...,-0.173944,0.065542,-0.112298,0.058498,0.003030,-0.006853,0.147340,-0.144262,0.004296,0.056279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618,NDARINVKR1PVY4C,0.034540,-0.122531,0.044441,-0.016456,-0.045248,-0.117800,-0.041358,0.019584,0.125240,...,0.030459,0.050246,0.105606,0.063260,-0.132830,-0.224092,-0.275648,0.268601,0.011595,-0.178984
3619,NDARINVKWG00F0U,-0.005203,0.027012,-0.071404,0.002445,-0.010985,-0.040965,-0.022926,-0.043118,-0.054241,...,0.121185,-0.001371,-0.027949,0.059797,0.072735,0.000886,0.023277,0.046444,0.067837,-0.030901
3620,NDARINVKTN3FFWD,0.086599,0.097084,0.025272,0.033580,-0.060310,-0.019461,-0.016505,-0.046254,0.049006,...,0.100596,-0.046986,0.099617,0.091862,0.062683,0.028582,0.027145,0.165274,0.061116,-0.013563
3621,NDARINVL42FE3NR,-0.007273,-0.022607,0.022100,0.061466,0.019361,0.025705,0.032865,0.048510,-0.020663,...,0.121750,-0.057892,0.000277,0.034063,-0.080776,-0.039805,-0.103092,-0.085185,0.005517,-0.033133
